# NON DEFOLIATED FIRES

This notebook identifies fires that did not have history of defoliation within 15 years prior to fire.

1. Identify fires
2. Get Indices
3. Process Indices
4. Get Host species
5. Get Weather



## Step 1. Identify fires


Load modules


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np



Load data 

In [6]:
# occurrence data

on_occurrences = pd.read_csv('../data/outputs/on/on_co-occurrence_defoliation_history.csv')
qc_occurrences = pd.read_csv('../data/outputs/qc/qc_co-occurrence_defoliation_history.csv')

# perimeter data
on_perimeter = gpd.read_file('../data/inputs/on/fire/ON_FirePerimeters_85to2020_v00.shp')
qc_perimeter = gpd.read_file('../data/inputs/qc/qc-fire-perims-shield-2.shp')

# 

Inspect column and dataframe structure

In [8]:
print("on_occurrences", on_occurrences.info())
print("qc_occurrences", qc_occurrences.info())
print("on_perimeters", on_perimeter.info())
print("qc_perimeters", qc_perimeter.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fire_ID                 365 non-null    object 
 1   Fire_Year               365 non-null    int64  
 2   Time_Since_Defoliation  365 non-null    int64  
 3   Cumulative_Years        365 non-null    int64  
 4   Max_Overlap_Area        365 non-null    float64
 5   Max_Overlap_Percent     365 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 17.2+ KB
on_occurrences None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fire_ID                 1009 non-null   object 
 1   Fire_Year               1009 non-null   int64  
 2   Time_Since_Defoliation  1009 non-null   float64
 3   Cumulative_Ye

Filter all fire perimeters that are not in the occurrence dataframe by Fire_ID

In [9]:
def filter_geodf_by_fire_id(geo_df, pandas_df):
    """
    Filters a GeoPandas DataFrame, keeping only the rows that do not have a value in the Fire_ID column
    that matches a value in the Fire_ID column of a Pandas DataFrame.

    Parameters:
    geo_df (gpd.GeoDataFrame): The GeoPandas DataFrame to filter.
    pandas_df (pd.DataFrame): The Pandas DataFrame containing Fire_ID values to exclude.

    Returns:
    gpd.GeoDataFrame: The filtered GeoPandas DataFrame.
    """
    # Get the set of Fire_ID values to exclude
    exclude_fire_ids = set(pandas_df['Fire_ID'])

    # Filter the GeoPandas DataFrame
    filtered_geo_df = geo_df[~geo_df['Fire_ID'].isin(exclude_fire_ids)]

    return filtered_geo_df

In [11]:
on_filtered_perimeters = filter_geodf_by_fire_id(on_perimeter, on_occurrences)
on_filtered_perimeters.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1257 entries, 0 to 1621
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OGF_ID      1257 non-null   int64   
 1   r_id        1257 non-null   object  
 2   raster_id   1257 non-null   object  
 3   Fire_ID     1257 non-null   object  
 4   FIRE_DISTU  1257 non-null   object  
 5   FIRE_TYPE_  1257 non-null   object  
 6   Fire_Year   1257 non-null   int64   
 7   FIRE_GENER  1257 non-null   object  
 8   FIRE_RESPO  1257 non-null   object  
 9   FIRE_START  1257 non-null   object  
 10  FIRE_OUT_D  1257 non-null   object  
 11  FIRE_FINAL  1257 non-null   int64   
 12  LOCATION_A  1257 non-null   object  
 13  BUSINESS_E  1094 non-null   object  
 14  GENERAL_CO  689 non-null    object  
 15  GEOMETRY_U  371 non-null    object  
 16  EFFECTIVE_  1257 non-null   object  
 17  SYSTEM_DAT  1257 non-null   object  
 18  OBJECTID    1257 non-null   int64   
 19  SHA

In [12]:
qc_filtered_perimeters = filter_geodf_by_fire_id(qc_perimeter, qc_occurrences)
qc_filtered_perimeters.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1220 entries, 21 to 2236
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Fire_ID    1220 non-null   object  
 1   prov       1220 non-null   object  
 2   nfireid    1220 non-null   object  
 3   Fire_Year  1220 non-null   object  
 4   BASRC      1220 non-null   float64 
 5   FIREMAPS   1220 non-null   float64 
 6   FIREMAPM   1220 non-null   float64 
 7   FIRECAUS   1220 non-null   float64 
 8   BURNCLAS   1220 non-null   float64 
 9   SDATE      276 non-null    object  
 10  EDATE      276 non-null    object  
 11  AFSDATE    950 non-null    object  
 12  AFEDATE    1017 non-null   object  
 13  CAPDATE    212 non-null    object  
 14  POLY_HA    1220 non-null   float64 
 15  ADJ_HA     1220 non-null   float64 
 16  ADJ_FLAG   1220 non-null   float64 
 17  BT_GID     1220 non-null   float64 
 18  VERSION    1220 non-null   object  
 19  COMMENTS   1040 non-nul

Save filtered geodataframes


In [13]:
on_filtered_perimeters.to_file('../data/outputs/on/on_no_insect_history_perimeters.shp')
qc_filtered_perimeters.to_file('../data/outputs/qc/qc_no_insect_history_perimeters.shp')

# Step 2. Get Indices

load required modules

In [14]:
import ee 
import geemap
import pandas as pd
from ltgee import LandTrendr, LandsatComposite, LtCollection
from datetime import date

intialize earth engine

In [15]:
ee.Initialize()

#### landtrendr functions

rename bands in image

In [16]:
def rename_bands(image):
    """
    Renames each band in the image by stripping "yr" from the name.

    Parameters:
    image (ee.Image): The input image with bands to be renamed.

    Returns:
    ee.Image: The image with renamed bands.
    """
    # Get the list of band names
    band_names = image.bandNames()

    # Create a list of new band names by stripping "yr" from each name
    new_band_names = band_names.map(lambda name: ee.String(name).replace('yr', '', 'g'))

    # Rename the bands in the image
    renamed_image = image.rename(new_band_names)

    return renamed_image

    

calculate burn severity metrics 

In [17]:
def calcBS(img, ft1):
    ft1 = ee.Feature(ft1)
    
    # Select the bands by their indices
    preNBR = img.select([0]).rename('preNBR')
    postNBR = img.select([2]).rename('postNBR')
    
    # Calculate dNBR
    dnbr = preNBR.subtract(postNBR).multiply(1000).rename('dnbr').toFloat()
    
    # Create a ring buffer around the feature
    ring = ft1.buffer(180).difference(ft1)
    
    # Calculate the offset
    offset = ee.Image.constant(ee.Number(dnbr.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ring.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('dnbr'))).rename('offset').toFloat()
    
    # Add the offset and original bands to the dNBR image
    dnbr = dnbr.addBands(offset).addBands(preNBR).addBands(postNBR)
    
    # Calculate dNBR with offset
    dnbr_offset = dnbr.expression("b('dnbr') - b('offset')").rename('dnbr_w_offset').toFloat()
    dnbr_offset = dnbr_offset.addBands(dnbr)
    
    # Calculate RBR
    rbr = dnbr_offset.expression("b('dnbr') / (b('preNBR') + 1.001)").rename('rbr').toFloat().addBands(dnbr_offset)
    
    # Calculate RBR with offset
    rbr_offset = rbr.expression("b('dnbr_w_offset') / (b('preNBR') + 1.001)").rename('rbr_w_offset').toFloat().addBands(rbr)
    
    # Set properties
    rbr_offset = rbr_offset.set('fireID', ft1.get('Fire_ID'), 'fireName', ft1.get('Fire_Name'), 'fireYear', ft1.get('Fire_Year'))
    
    converted_to_float = rbr_offset.toFloat()

    return converted_to_float



full landtrendr function for a given feature 

In [28]:
def export_landtrendr_image(feature):

    # get the feature's ID, geometry and year
    fire_id = feature.get('Fire_ID').getInfo()
    geometry = feature.geometry().bounds()
    fire_year = feature.get('Fire_Year').getInfo()
    print(f"fire check {fire_year}")

    if fire_id is None or fire_year is None:
        print(f"Skipping fire with missing Fire_ID or Fire_Year: {fire_id}, {fire_year}")
        return

    # turn fire year into a string and get prefire year
    fire_year = str(fire_year)
    prefire_year = str(int(fire_year) - 1)
    postfire_year = str(int(fire_year) + 11)

    # Example arguments to pass to the LandTrendr class. See docstring of LandTrendr for more information'
    composite_params = {
    "start_date": date(int(prefire_year), 4,1),
    "end_date": date(int(postfire_year), 11,1),
    "area_of_interest": geometry,
    "mask_labels": ['cloud', 'shadow', 'snow', 'water'],
    "debug": True
}
    lt_collection_params = {
        "sr_collection": LandsatComposite(**composite_params),
        # "sr_collection": composite_params, # - you may also just pass in your own collection or the params directly
        "index": 'NBR',
        "ftv_list": ['NBR']}
    lt_params = {
        "lt_collection": LtCollection(**lt_collection_params),
        # "lt_collection": lt_collection_params, # - you may also just pass in your own collection or the params directly
        "run_params": {
                "maxSegments": 6,
                "spikeThreshold": 0.9,
                "vertexCountOvershoot":  3,
                "preventOneYearRecovery":  True,
                "recoveryThreshold":  0.25,
                "pvalThreshold":  .05,
                "bestModelProportion":  0.75,
                "minObservationsNeeded": 2,
            }
    }

    lt = LandTrendr(**lt_params)
    lt.run()
    
    ftv_nbr = lt.select('ftv_nbr_fit').clip(composite_params['area_of_interest'])

    years = []

    # Iterate over the range of years and append the formatted strings to the list
    for i in range(int(prefire_year), int(postfire_year) + 1):
        years.append('yr' + str(i))


    # convert 1-D fitted nbr array to an image with bands representing years
    nbr_stack = ftv_nbr.arrayFlatten([years])

    # clean the names of the nbr stack bands
    clean_nbr_stack = rename_bands(nbr_stack)

    # calculate burn severity indices
    results = calcBS(clean_nbr_stack, feature)

    # make the file names to export each image
    export_bi_name = fire_id + "_bi"
    export_nbr_name = fire_id + "_nbr"
    
    # export  burn Indices imagery
    geemap.ee_export_image_to_drive(results, description=export_bi_name , folder="chapter3_lt_bi_v2_qc_no_history", region=geometry, scale=30)

    # export nbr recovery imagery
    geemap.ee_export_image_to_drive(clean_nbr_stack, description=export_nbr_name, folder="chapter3_lt_nbr_v2_qc_no_history", region=geometry, scale=30)
    


    



#### Processing Workflow


##### ontario

filter the gee feature collection to select only the fires with no insect history

In [ ]:
# read in fire-insect co-occurrence data
occurrence = gpd.read_file('/home/goldma34/fire_insect_co-occurence/data/outputs/on/on_no_insect_history_perimeters.shp')

# get fire names and turn it into a list
fire_names = occurrence['Fire_ID'].unique().tolist()

# Read in fires from GEE
# Filter the fires by the names in the list
fires = ee.FeatureCollection("users/jandrewgoldman/Ont_BurnSeverity_Trends/ON_FirePerimeters_85to2020_v0").filter(ee.Filter.inList('Fire_ID', fire_names))


convert the Fire_ID column to a list to be iterated over in a for loop

In [20]:
fire_ids = fires.aggregate_array('Fire_ID').getInfo()

iterate over each Fire_ID and run the landtrendr algorithm 

In [29]:

for fire_id in fire_ids:
        print(fire_id)
        fire = fires.filterMetadata('Fire_ID', 'equals', fire_id).first()
        results = export_landtrendr_image(fire)
        print(f"Exported LandTrendr imagery for fire {fire_id}")


SLK30_2006_1581
fire check 2006
Exported LandTrendr imagery for fire SLK30_2006_1581
SLK49_2010_1730
fire check 2010
Exported LandTrendr imagery for fire SLK49_2010_1730
NIP38_2015_1995
fire check 2015
Exported LandTrendr imagery for fire NIP38_2015_1995
RED7_1986_119
fire check 1986
Exported LandTrendr imagery for fire RED7_1986_119
GER14_1986_340
fire check 1986
Exported LandTrendr imagery for fire GER14_1986_340
RED5_1986_368
fire check 1986
Exported LandTrendr imagery for fire RED5_1986_368
GER29_1986_385
fire check 1986
Exported LandTrendr imagery for fire GER29_1986_385
GER13_1986_576
fire check 1986
Exported LandTrendr imagery for fire GER13_1986_576
GER12_1986_801
fire check 1986
Exported LandTrendr imagery for fire GER12_1986_801
RED24_1986_950
fire check 1986
Exported LandTrendr imagery for fire RED24_1986_950
RED14_1986_1055
fire check 1986
Exported LandTrendr imagery for fire RED14_1986_1055
SIO91_1986_1158
fire check 1986
Exported LandTrendr imagery for fire SIO91_1986_115

KeyboardInterrupt: 

##### Quebec



In [25]:
# read in fire-insect co-occurrence data
occurrence_qc = gpd.read_file('/home/goldma34/fire_insect_co-occurence/data/outputs/qc/qc_no_insect_history_perimeters.shp')

# get fire names and turn it into a list
fire_names_qc = occurrence_qc['Fire_ID'].unique().tolist()

# Read in fires from GEE
# Filter the fires by the names in the list
fires_qc = ee.FeatureCollection("users/jandrewgoldman/qc-fire-perims-shield-2").filter(ee.Filter.inList('Fire_ID', fire_names_qc))


In [26]:
fire_ids_qc = fires_qc.aggregate_array('Fire_ID').getInfo()

In [30]:
for fire_id in fire_ids_qc:
        print(fire_id)
        fire = fires_qc.filterMetadata('Fire_ID', 'equals', fire_id).first()
        results = export_landtrendr_image(fire)
        print(f"Exported LandTrendr imagery for fire {fire_id}")


QC_545_2012
fire check 2012
Exported LandTrendr imagery for fire QC_545_2012
QC_189_1996
fire check 1996
Exported LandTrendr imagery for fire QC_189_1996
QC_245_1996
fire check 1996
Exported LandTrendr imagery for fire QC_245_1996
QC_258_2005
fire check 2005
Exported LandTrendr imagery for fire QC_258_2005
QC_366_2006
fire check 2006
Exported LandTrendr imagery for fire QC_366_2006
QC_296_2008
fire check 2008
Exported LandTrendr imagery for fire QC_296_2008
QC_805_2013
fire check 2013
Exported LandTrendr imagery for fire QC_805_2013
QC_613_2014
fire check 2014
Exported LandTrendr imagery for fire QC_613_2014
QC_1941_2014
fire check 2014
Exported LandTrendr imagery for fire QC_1941_2014
QC_364_2009
fire check 2009
Exported LandTrendr imagery for fire QC_364_2009
QC_1179_2015
fire check 2015
Exported LandTrendr imagery for fire QC_1179_2015
QC_928_1998
fire check 1998
Exported LandTrendr imagery for fire QC_928_1998
QC_223_1999
fire check 1999
Exported LandTrendr imagery for fire QC_223_